In [10]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re
import time

# brew 로 설치된 chromedriver의 path (Mac)
# path = '/usr/local/bin/chromedriver'

# 윈도우용 크롬 웹드라이버 실행 경로 (Windows)
path = 'chromedriver.exe'

# 크롤링할 사이트 주소 정의
source_url = "https://map.kakao.com/"

# 크롬 드라이버 사용
# driver = webdriver.Chrome(path)
driver = webdriver.Chrome(executable_path=excutable_path)

# 카카오 지도에 접속
driver.get(source_url)

# 검색
driver.find_element(By.XPATH, "//input[@id='search.keyword.query']").send_keys("가디역 맛집")
searchbutton = driver.find_element(By.XPATH, "//button[@id='search.keyword.submit']")
driver.execute_script("arguments[0].click();", searchbutton)

# 검색 결과를 가져올 시간을 기다림
time.sleep(2)

html = driver.page_source

# html 정보 파싱
soup = BeautifulSoup(html, "html.parser")
moreviews = soup.find_all(name="a", attrs={"class":"moreview"})

# a태그의 href 속성을 리스트로 추출하여, 크롤링 할 페이지 리스트를 생성
page_urls = []
for moreview in moreviews:
    page_url = moreview.get("href")
    print(page_url)
    page_urls.append(page_url)

# 브라우저 종료
driver.close()

https://place.map.kakao.com/1954563179
https://place.map.kakao.com/25449227
https://place.map.kakao.com/929624344
https://place.map.kakao.com/26451376
https://place.map.kakao.com/27466069
https://place.map.kakao.com/1652547690
https://place.map.kakao.com/16878201
https://place.map.kakao.com/1549878166
https://place.map.kakao.com/2074600451
https://place.map.kakao.com/1305442271
https://place.map.kakao.com/2062610369
https://place.map.kakao.com/918167314
https://place.map.kakao.com/14547125
https://place.map.kakao.com/24926523
https://place.map.kakao.com/2020851028


In [59]:
columns = ['score', 'review']
df = pd.DataFrame(columns=columns)

# driver = webdriver.Chrome(path)
driver = webdriver.Chrome(executable_path=path)

for page_url in page_urls:
    
    driver.get(page_url) # 상세 페이지
    time.sleep(2)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    contents_div = soup.find(name='div', attrs={'class':'evaluation_review'}) 
    if(isinstance(contents_div, type(None))):
        continue
    rates = contents_div.find_all(name='span', attrs={'class':'inner_star'}) # 별점
    reviews = contents_div.find_all(name='p', attrs={'class':'txt_comment'}) # 리뷰
    
    for rate, review in zip(rates, reviews):
        row = [rate["style"].split(":")[1][:-2], review.find(name='span').text]
        series = pd.Series(row, index=df.columns)
        df = df.append(series, ignore_index=True)

driver.close()

In [60]:
df

,score,review
0,100,맛있었어요콩나물 엄청 좋아해서 많이 먹었는데도 많이 남았어요 조금 아까웠슴 ㅠㅠ남은...
1,100,맛있음 신라면도 매워서 못먹는 맵찔이는 비추 덜매운맛과 매운맛만 존재함
2,40,1. 자리 원하는 곳 못않게 함(단체석이라더니 결국 아니었음...)2. 맛은 sos...
3,100,넘 맛있어용 대기줄 길고 내부는 평범한 고깃집이라 환기는 좀 안되네요 ㅎㅎ
4,100,여기 맛있습니다
5,100,고기에 칼집+초벌구이해서 나와요. 정말 부드럽고 맛있습니다. 날씨좋으면 야외에서 먹...
6,100,너어무 맛있었어요! 군만두 마파두부 볶음밥 지금도 생각나요…
7,100,저렴하고 너무 맛있습니다. 하얼빈과 함께먹은 육즙터지는 만두의맛... 마파두부도 맛...
8,100,내가 먹어본 만두중 가장 중국만두에 가까운만두 친~차이만두.
9,80,미션: 김치의 유혹에서 벗어나시오 따라나오는 국수 유혹에서도 벗어나시오


In [61]:
# 4점 이상의 리뷰는 긍정 리뷰, 3점 이하의 리뷰는 부정 리뷰로 평가
df['y'] = df['score'].apply(lambda x: 1 if float(x) > 60 else 0)
print(df.shape)
df.head()

(42, 3)


,score,review,y
0,100,맛있었어요콩나물 엄청 좋아해서 많이 먹었는데도 많이 남았어요 조금 아까웠슴 ㅠㅠ남은...,1
1,100,맛있음 신라면도 매워서 못먹는 맵찔이는 비추 덜매운맛과 매운맛만 존재함,1
2,40,1. 자리 원하는 곳 못않게 함(단체석이라더니 결국 아니었음...)2. 맛은 sos...,0
3,100,넘 맛있어용 대기줄 길고 내부는 평범한 고깃집이라 환기는 좀 안되네요 ㅎㅎ,1
4,100,여기 맛있습니다,1


In [62]:
df.to_csv("review_data.csv", index=False)